# <center> Modelo Estocástico <center>
    
    Aplicaremos el modelo de Mack, que es estocástico para predecir las futuras reclamaciones basados en los valores históricos de los triángulos acumulados de pérdidas y estimaremos el error estándar alrededor de estos valores.
   Las funciones que utilizaremos ya se encuentran implementadas en módulo chainladder de python

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display, HTML
# !(pip install chainladder)
import chainladder as cl # librería para realizar triángulos de pérdidas
import statsmodels.api as sm

print("pandas: " + pd.__version__)
print("numpy: " + np.__version__)
print("chainladder: " + cl.__version__)

pandas: 1.4.2
numpy: 1.22.0
chainladder: 0.8.18


## Ingreso de los Datos

In [2]:
data=pd.read_csv('wkcomp_pos.csv') # cargar o abrir un csv
display(data.head())

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
0,86,Allstate Ins Co Grp,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872
1,86,Allstate Ins Co Grp,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872
2,86,Allstate Ins Co Grp,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872
3,86,Allstate Ins Co Grp,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872
4,86,Allstate Ins Co Grp,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872


Creamos un triángulo con los datos de entrada, para el análisis vamos a tomar únicamente las variables de pérdida. Este triangulo es el triángulo de la variable CumPaidLoss_D sumado para todas las compañías listadas, por lo cual se obtiene un solo triángulo total.

In [3]:
triangle = cl.Triangle(
    data,
    origin="AccidentYear", # filas
    development="DevelopmentYear", # columnas
    columns=data.columns[5:], # tomamos en cuenta solamente las columnas de ganancias y pérdidas
    index=["GRNAME"], # nombres de las compañías
    cumulative=False, # triángulo de pérdidas acumulado o no
)['CumPaidLoss_D'].sum()
#objeto de pandas
triangle
# triangle['CumPaidLoss_D'].sum() 

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228
1988,"285,804","638,532","865,100","996,363","1,084,351","1,133,188","1,169,749","1,196,917","1,229,203","1,241,715",,,,,,,,,
1989,"307,720","684,140","916,996","1,065,674","1,154,072","1,210,479","1,249,886","1,291,512","1,308,706","1,324,671",,,,,,,,,
1990,"320,124","757,479","1,017,144","1,169,014","1,258,975","1,315,368","1,368,374","1,394,675","1,417,384","1,431,483",,,,,,,,,
1991,"347,417","793,749","1,053,414","1,209,556","1,307,164","1,381,645","1,414,747","1,447,121","1,465,508","1,479,177",,,,,,,,,
1992,"342,982","781,402","1,014,982","1,172,915","1,281,864","1,328,801","1,370,935","1,399,901","1,419,809","1,437,891",,,,,,,,,
1993,"342,385","743,433","959,147","1,113,314","1,187,581","1,243,689","1,278,194","1,301,968","1,322,101","1,337,171",,,,,,,,,
1994,"351,060","750,392","993,751","1,114,842","1,193,861","1,243,285","1,276,145","1,302,620","1,321,778","1,340,950",,,,,,,,,
1995,"343,841","768,575","962,081","1,087,925","1,164,217","1,210,269","1,243,983","1,264,903","1,282,868","1,300,678",,,,,,,,,
1996,"381,484","736,040","937,936","1,056,949","1,131,168","1,182,716","1,218,813","1,239,839","1,260,284","1,274,282",,,,,,,,,
1997,"340,132","730,838","940,850","1,066,652","1,142,476","1,191,886","1,223,282","1,248,906","1,262,903","1,277,512",,,,,,,,,


In [4]:
# triangle=triangle[triangle.development <= 120][triangle.origin < "1997"]
triangle

,12,24,36,48,60,72,84,96,108,120,132,144,156,168,180,192,204,216,228
1988,"285,804","638,532","865,100","996,363","1,084,351","1,133,188","1,169,749","1,196,917","1,229,203","1,241,715",,,,,,,,,
1989,"307,720","684,140","916,996","1,065,674","1,154,072","1,210,479","1,249,886","1,291,512","1,308,706","1,324,671",,,,,,,,,
1990,"320,124","757,479","1,017,144","1,169,014","1,258,975","1,315,368","1,368,374","1,394,675","1,417,384","1,431,483",,,,,,,,,
1991,"347,417","793,749","1,053,414","1,209,556","1,307,164","1,381,645","1,414,747","1,447,121","1,465,508","1,479,177",,,,,,,,,
1992,"342,982","781,402","1,014,982","1,172,915","1,281,864","1,328,801","1,370,935","1,399,901","1,419,809","1,437,891",,,,,,,,,
1993,"342,385","743,433","959,147","1,113,314","1,187,581","1,243,689","1,278,194","1,301,968","1,322,101","1,337,171",,,,,,,,,
1994,"351,060","750,392","993,751","1,114,842","1,193,861","1,243,285","1,276,145","1,302,620","1,321,778","1,340,950",,,,,,,,,
1995,"343,841","768,575","962,081","1,087,925","1,164,217","1,210,269","1,243,983","1,264,903","1,282,868","1,300,678",,,,,,,,,
1996,"381,484","736,040","937,936","1,056,949","1,131,168","1,182,716","1,218,813","1,239,839","1,260,284","1,274,282",,,,,,,,,
1997,"340,132","730,838","940,850","1,066,652","1,142,476","1,191,886","1,223,282","1,248,906","1,262,903","1,277,512",,,,,,,,,


Creación del modelo

In [5]:
mack = cl.MackChainladder().fit(triangle)

    Vamos a tomar las primeras dos columnas, correspondientes a los triángulos en los meses 12 y 24

In [6]:
clrd_first_lags = triangle[triangle.development <= 24][triangle.origin < "1997"]["CumPaidLoss_D"]
clrd_first_lags

,12,24
1988,"285,804","638,532"
1989,"307,720","684,140"
1990,"320,124","757,479"
1991,"347,417","793,749"
1992,"342,982","781,402"
1993,"342,385","743,433"
1994,"351,060","750,392"
1995,"343,841","768,575"
1996,"381,484","736,040"


La  columna 1 es la variable dependiente y las primera columna sirve de variable independiente,
es decir el comportamiento del período siguiente depende exclusivamente del anterior. Sobre estos datos se realizará un ajuste por mínimos cuadrados (realizado con sm.WLS), solo que ahora los datos tendrán un peso $1/x^2$.

In [7]:
y = clrd_first_lags.to_frame(origin_as_datetime=True).values[:, 1]
x = clrd_first_lags.to_frame(origin_as_datetime=True).values[:, 0]

model = sm.WLS(y, x, weights=(1 / x) ** 2)
results = model.fit()
results.summary()

/home/john/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 WLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.997
Model:                            WLS   Adj. R-squared (uncentered):              0.997
Method:                 Least Squares   F-statistic:                              2887.
Date:                Mon, 04 Dec 2023   Prob (F-statistic):                    1.60e-11
Time:                        18:11:51   Log-Likelihood:                         -107.89
No. Observations:                   9   AIC:                                      217.8
Df Residuals:                       8   BIC:                                      218.0
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             2.2067      0.041     53.735      0.000       2.112       2.301
==============================================================================
Omnibus:                        7.448   Durbin-Watson:                   1.177
Prob(Omnibus):                  0.024   Jarque-Bera (JB):                2.533
Skew:                          -1.187   Prob(JB):                        0.282
Kurtosis:                       4.058   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
dev = cl.Development(average="simple").fit(triangle["CumPaidLoss_D"])
print("Sigma")
display(dev.sigma_)
print("Error Estándar")
display(dev.std_err_)

Sigma


,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120,120-132,132-144,144-156,156-168,168-180,180-192,192-204,204-216,216-228
(All),0.1176,0.0334,0.0156,0.0094,0.0057,0.0040,0.0033,0.0027,0.0019,,,,,,,,,


Error Estándar


,12-24,24-36,36-48,48-60,60-72,72-84,84-96,96-108,108-120,120-132,132-144,144-156,156-168,168-180,180-192,192-204,204-216,216-228
(All),0.0372,0.0106,0.0049,0.0030,0.0018,0.0013,0.0011,0.0008,0.0006,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


## Resumen de las predicciones del modelo para el último período

In [9]:
mack.summary_

,Latest,IBNR,Ultimate,Mack Std Err
1988,,,"9,840,922",
1989,,,"10,513,856",
1990,,,"11,450,020",
1991,,,"11,899,498",
1992,,,"11,551,482",
1993,,,"10,828,983",
1994,,,"10,888,684",
1995,,,"10,629,340",
1996,,,"10,419,511",
1997,"1,277,512",,"10,425,437",


* Interpretación

**R-cuadrado y R-cuadrado Ajustado:**
   - El valor de R-cuadrado es 0.997, lo que indica que aproximadamente el 99.7% de la varianza en la variable dependiente (y) es explicada por las variables independientes. Este valor sugiere un ajuste muy bueno.
   - El R-cuadrado ajustado también es 0.997, lo que significa que ajusta el R-cuadrado para tener en cuenta el número de variables independientes en el modelo.

**Información del Modelo:**
   - Modelo: Especifica que es un modelo WLS.
   - Método: Mínimos Cuadrados.
   - El estadístico F es 2887, y el valor p asociado es 1.60e-11, lo que sugiere que el modelo es estadísticamente significativo.

**Observaciones y Grados de Libertad:**
   - No. Observaciones: El número de puntos de datos en el conjunto de datos es 9.
   - Df Residuals: Grados de libertad de los residuos son 8 (la diferencia entre el número de observaciones y el número de parámetros en el modelo).
   - Df Modelo: Grados de libertad del modelo son 1 (el número de variables independientes).

6. **Tipo de Covarianza:**
   - El tipo de covarianza utilizado en la estimación es no robusta.

7. **Coeficientes:**
   - El coeficiente (coef) para la variable independiente x1 es 2.2067.
   - El error estándar (std err) asociado a este coeficiente es 0.041.
   - El estadístico t (t) es 53.735, y el valor p asociado (P>|t|) es prácticamente cero (0.000), lo que indica que el coeficiente x1 es estadísticamente significativo.
   - El intervalo de confianza del 95% ([0.025 0.975]) para el coeficiente x1 es (2.112, 2.301).

10. **IBNR, Ultimate, Mack Std Err:**
    - Los valores específicos para IBNR, Ultimate, y Mack Std Err no están proporcionados en la descripción.

En resumen, el modelo de regresión parece tener un ajuste muy sólido con un R-cuadrado alto. La variable independiente x1 tiene un coeficiente significativo y la estadística F respalda la significancia general del modelo. Las estadísticas adicionales proporcionan información sobre la normalidad y ajuste de los residuos.